[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/inferencebot-colab/blob/main/inferencebot_mistral_7b_instruct_v0.2_colab.ipynb)

In [ ]:
%cd /content

!wget https://go.dev/dl/go1.21.5.linux-amd64.tar.gz
!tar -C /content -xzf go1.21.5.linux-amd64.tar.gz
import os
os.environ["PATH"] = os.environ.get("PATH", "") + ":/content/go/bin"
os.environ["GOPATH"] = os.path.expanduser("~/go")
os.environ["PATH"] = os.environ.get("PATH", "") + ":" + os.path.join(os.environ["GOPATH"], "bin")
!wget https://github.com/camenduru/inferencebot/raw/main/index.html -O /content/index.html
!wget https://github.com/camenduru/inferencebot/raw/main/main.go -O /content/main.go
!go mod init github.com/camenduru/websocket-echo-server
!go get github.com/gorilla/websocket
!go get github.com/gempir/go-twitch-irc/v3

!apt install -y -qq xvfb libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev gstreamer1.0-plugins-good gstreamer1.0-plugins-bad gstreamer1.0-plugins-ugly gstreamer1.0-tools gstreamer1.0-libav
!wget https://huggingface.co/camenduru/cefsrc/resolve/main/cefsrc.zip
!unzip /content/cefsrc.zip -d /content/cefsrc

In [ ]:
!pip install -q accelerate bitsandbytes

from typing import Iterator
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

import gradio as gr

def generate(prompt):
  messages = [{"role": "user", "content": prompt},]
  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = encodeds.to(model.device)
  generated_ids = model.generate(model_inputs, max_new_tokens=1024, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  import re
  text = re.sub(r'\[INST\].*?\[/INST\]', '', decoded[0])
  return text

with gr.Blocks(title=f"mistral-7b-instruct-v0.2", css=".gradio-container {max-width: 544px !important}") as demo:
  with gr.Row():
    with gr.Column():
        textbox = gr.Textbox(show_label=False, value="who are you?")
        button = gr.Button()
  with gr.Row(variant="default"):
      output_text = gr.Textbox(show_label=False)

  button.click(fn=generate, inputs=[textbox], outputs=[output_text], show_progress=True)

demo.queue().launch(inline=False, share=True, debug=False)

In [ ]:
username = "username_sed" #@param {type: 'string'}
username_code = f's/username_sed/{username}/g'
channel = "channel_sed" #@param {type: 'string'}
channel_code = f's/channel_sed/{channel}/g'
oauth = "oauth_sed" #@param {type: 'string'}
oauth_code = f's/oauth_sed/{oauth}/g'
!sed -i -e {username_code} -e {channel_code} -e {oauth_code} main.go

import os
import subprocess
import threading

def set_environment_variables():
    os.environ["PATH"] = os.environ.get("PATH", "") + ":/content/go/bin"
    os.environ["GOPATH"] = os.path.expanduser("~/go")
    os.environ["PATH"] = os.environ.get("PATH", "") + ":" + os.path.join(os.environ["GOPATH"], "bin")

def run_go_program():
    try:
        subprocess.run(["go", "run", "."], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running Go program: {e}")

def main():
    set_environment_variables()
    go_thread = threading.Thread(target=run_go_program)
    go_thread.start()
    # go_thread.join()

if __name__ == "__main__":
    main()

In [ ]:
%env LD_PRELOAD=/content/cefsrc/libcef.so
rtmp_link = "rtmp_link" #@param {type: 'string'}
stream_code = f"-e /dev/stdout gst-launch-1.0 cefsrc url=http://127.0.0.1 ! video/x-raw,format=BGRA,width=960,height=540,framerate=5/1 ! cefdemux name=demux ! videoconvert ! video/x-raw,format=I420 ! videoconvert ! queue ! x264enc speed-preset=ultrafast tune=zerolatency key-int-max=60 ! flvmux name=muxer ! rtmpsink location={rtmp_link} demux. ! liveadder ! avenc_aac bitrate=128000 ! muxer."
%env GST_PLUGIN_PATH=/content/cefsrc:$GST_PLUGIN_PATH 
!xvfb-run {stream_code}